### Importação das bibliotecas

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Importação do CSV (Data de última atualização deste dataset")

In [2]:
csv_path = 'data/Manaus.csv'
#csv_path = 'csv_path = 'https://covid19.manaus.am.gov.br/wp-content/uploads/Manaus.csv' Realtime dataset
df = pd.read_csv(csv_path,encoding='iso-8859-1',sep=';')

/home/yonekura/anaconda3/envs/paic/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12,15,22,24,30,31,32,33,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2. Visão geral dos casos confirmados

### 1. Considerar apenas casos confirmados

In [3]:
df = df[df._classificacao.eq('Confirmado')]

### 2. Quantos atributos descrevem cada exemplo? Quais são eles?


In [4]:
print(f"São {len(df.columns)} atributos, sendo eles:")
print(df.columns)

São 36 atributos, sendo eles:
Index(['_idade', '_faixa etária', '_sexo', '_bairro', '_classificacao',
       '_comorb_renal', '_comorb_diabetes', '_comorb_imuno', '_comorb_cardio',
       '_conclusao', '_dt_notificacao', '_taxa', '_dt_evolucao', '_raca',
       '_dt_sintomas', '_criterio', '_tipo_teste', '_sintoma_garganta',
       '_sintoma_dispneia', '_sintoma_febre', '_sintoma_tosse',
       '_sintoma_outros', '_etnia', '_profiss_saude', '_srag',
       '_se_notificacao', '_distrito', '_bairro_mapa', '_comorb_respiratoria',
       '_comorb_cromossomica', '_comorb_hepatica', '_comorb_neurologica',
       '_comorb_hemato', '_comorb_obessidade', '_origem', '_evolução'],
      dtype='object')


### 3. Quantos casos confirmados há em Manaus, cumulativamente?

In [5]:
print(f"Segundo o dataset utilizado, há atualmente {len(df)} casos confirmados em Manaus")

Segundo o dataset utilizado, há atualmente 36947 casos confirmados em Manaus


### 4. A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o mais recente? Leve em conta a data de notificação.

In [6]:
#df["_dt_notificacao"] = pd.to_datetime(df["_dt_notificacao"])
df["_dt_notificacao"] = pd.to_datetime(df["_dt_notificacao"],format='%d/%m/%Y')

print(df.sort_values(['_dt_notificacao'])["_dt_notificacao"][:10])

107318   2020-01-30
76875    2020-02-16
77211    2020-02-16
99336    2020-02-23
49916    2020-03-06
89016    2020-03-09
108094   2020-03-11
90875    2020-03-12
68700    2020-03-13
91078    2020-03-14
Name: _dt_notificacao, dtype: datetime64[ns]


Acima, convertemos o tipo de dados da coluna _dt_notificacao para DateTime

In [7]:
df["_dt_notificacao"].min()

Timestamp('2020-01-30 00:00:00')

In [8]:
df["_dt_notificacao"].max()

Timestamp('2020-08-06 00:00:00')

Usando as funções min e max, concluímos que o registro mais antigo é do dia 30/01/2020 e o mais recente é 06/08/2020;

Para que as seguintes questões fossem resolvidas, fora necessário a seleção dos atributos da base de dados

In [9]:
filterList = ['_comorb', 'sintoma(s)*','_etnia','_profiss_saude','^(_dt_.*(?<!notificacao)$)','_origem', 
              '_raca', '_criterio','_srag','_evolução']
for i in range(len(filterList)):
    df = df[df.columns.drop(list(df.filter(regex=filterList[i])))]
dfWithTestType = df.copy()
df = df[df.columns.drop(list(df.filter(regex='_tipo_teste')))]
df.dropna(inplace=True)
dfWithTestType.dropna(inplace=True)

### 1) Quantos exemplos e atributos há na base de dados após a limpeza e organização?

In [38]:
print("Total de casos confirmados após os filtros: ", len(df.index))
print("Total de colunas após os filtros: ",len(df.columns))

Total de casos confirmados após os filtros:  12672
Total de colunas após os filtros:  11


### 2) Qual a porcentagem de indivíduos recuperados em relação ao todo?

In [12]:
percentRecovered = len(df[df._conclusao.eq('Recuperado')])/len(df.index)
print("Porcentagem de individuos recuperados em relação ao todo após os filtros: %.3f" % (percentRecovered*100))

Porcentagem de individuos recuperados em relação ao todo após os filtros: 83.917


### 3) Os casos acometeram mais indivíduos do sexo masculino ou feminino?

In [13]:
print("Masculino: ", len(df[df._sexo.eq('M')]))
print("Feminino ", len(df[df._sexo.eq('F')]))

Masculino:  6367
Feminino  6305


### 4) Qual a média e desvio padrão de idade dos indivíduos que contraíram COVID-19? Qual o indivíduo mais jovem e o mais idoso a contraírem tal enfermidade?

In [14]:
print("Média da idade dos Confirmados é: %.3f" %df['_idade'].mean())
print("O desvio padrão da idade dos Confirmados é: %.3f" %df['_idade'].std())
print("A pessoa mais jovem a contrair tem: ", df['_idade'].min(), "anos, A mais idosa possui: ", df['_idade'].max())

Média da idade dos Confirmados é: 47.649
O desvio padrão da idade dos Confirmados é: 18.143
A pessoa mais jovem a contrair tem:  0.0 anos, A mais idosa possui:  118.0


### 5)Qual o bairro com maior incidência de casos?

In [ ]:
print("5) O bairro de maior incidencia de casos é: ", df['_bairro'].mode())

O bairro de maior incidencia de casos é:  0    CIDADE NOVA

### 6) Quais os três bairros com maior incidência de casos recuperados?

In [ ]:
df_conclusionRecovered = df[df['_conclusao'] == 'Recuperado']
higherRecoveredNeighbor = df_conclusionRecovered.groupby(['_bairro','_conclusao'])._conclusao.count().sort_values(ascending=False).iloc[0:3]
print("6) Os bairros com maior incidencia de casos recuperados são: \n", higherRecoveredNeighbor)

Os bairros com maior incidencia de casos recuperados são: 
 _bairro      _conclusao
CIDADE NOVA  Recuperado    521
FLORES       Recuperado    388
CENTRO       Recuperado    351

### 7. Quais os tipos de testes efetuados, segundo os dados? Indique os dados de maneira quantitativa e percentual.

In [17]:
testType = dfWithTestType['_tipo_teste'].value_counts()
percent = dfWithTestType['_tipo_teste'].value_counts(normalize=True) * 100
print('Quantitativo:')
print(testType)
print('\nPercentual:')
print(percent)

Quantitativo:
TESTE RÁPIDO - ANTICORPO    3602
RT-PCR                      1501
TESTE RÁPIDO - ANTÍGENO     1101
ELISA IgM                      6
ECLIA IgG                      4
Name: _tipo_teste, dtype: int64

Percentual:
TESTE RÁPIDO - ANTICORPO    57.965883
RT-PCR                      24.155134
TESTE RÁPIDO - ANTÍGENO     17.718056
ELISA IgM                    0.096556
ECLIA IgG                    0.064371
Name: _tipo_teste, dtype: float64


### 8. Qual taxa de letalidade pode ser calculada a partir do conjunto de dados? Para calcular esta taxa, considere a fração do total de óbitos pelo total de casos

In [18]:
conclusions = df['_conclusao'].value_counts()
total_cases = conclusions.sum()
deaths = conclusions['Óbito']
letality = deaths / total_cases
letality

0.1608270202020202

### 9. Qual o tipo de correlação, mediante coeficiente de correlação de Pearson, entre a idade e o número de casos? Para responder a esta pergunte, agrupe o número de casos por idade e efetue o cálculo de tal coeficiente. Indique, a partir do resultado, a natureza desta correlação, se é positiva ou negativa, e qual sua intensidade.

In [19]:
groupedByAge = df.loc[:]
groupedByAge = groupedByAge['_idade'].value_counts().reset_index()
groupedByAge.columns = ['idade', 'quantidade']
groupedByAge
groupedByAge.corr(method='pearson')

,idade,quantidade
idade,1.000000,-0.179869
quantidade,-0.179869,1.000000


O valor da correlação de Pearson é igual a -0.179869 e indica uma correlação linear desprezível entre idade e número de casos

## 2..3. Tipos de Tarefas

R1) 
Analisando a base de dados após as devidas reduções e filtragens de dados nota-se que para uma tarefa de Classificação mediante Aprendizado Supervisionado " _faixaetária" e "_conclusao" podem ser o atributo-alvo, pois um segue um padrão de a cada nove em nove anos, como 20 a 29 ou 30 a 39 anos e o outro se um pessoa foi Recuperada ou não.

A métrica de desempenho utilizada nesta tarefa é a matriz de confusão na qual utilizaremos esta matriz para realizar os calculos da precisão e da revocação e por fim calcular o F-Score para que assim seja possível avaliar o modelo.

A avaliação mais recomendada para as tarefas de Classificação mediante Aprendizado Supervisionado é a validação cruzada do tipo K-fold pois é nela que será garantido que o modelo irá treinar com todas as partições dos dados e que realizará testes em cima das mesmas.

R2)
Analisando os dados para uma tarefa de regressão é possivel obter mais de um atributo-alvo, pode-se utilizar a "_idade" e "_taxa, pois estes atributos é possiível predizer um valor aproximado de um número real.

Os atributos preditores relevantes para este problema são a "_faixaetária", a "_classificacao", "_conclusao", "_bairro","_data" e "_se_notificacao". Tais atributos podem vir a aprimorar a predição de algum atributo alvo, como por exemplo a taxa de infecção de um bairro em uma data futura.

R3)
Pode-se utilizar clusterização para definir um grupo de risco.

